### Improvements 
## IMP: Fitrar por la surface--remove events that move out of the surface on the first 3 secons
1) Remove blinks and interpolate values using cubic splines
3) Frequency analysis of the signal??
4) Remove trials with z scores over [2,-2]
7) Statistics of signal ¿How to compare surprise signals?
8) Amplitud, velocidad de cambio
9) Cut the whole signal of interest from the beggining
10) Fitrar por la surface




In [29]:
import pandas as pd 
import numpy as np
from pathlib import Path
import os
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import signal
import sys
sys.path.append('../')
import commons as cm
from data_curation import Normalization
# from pandasql import sqldf
import pupillometry_commons as cp
from itertools import compress
import json

nm=Normalization()

session_location = Path(r"D:/sessions")
sessions=[]
df_list=[]
confidence_thr=0.95 #pupil condifedence for data cleaning
windows_s=3      #Seconds od interest for the signal
seconds_norm=0.05 #Baseline period for normalization

# Columns to load for pupil data
use_cols = ['confidence',
            'method',
            'pupil_timestamp',
            'eye_id',
            'diameter_3d',
            'diameter']


with open('session.txt','r') as f:
    for session in f:
            sessions.append(session.replace('\n',''))
f.close()

session_loc_list=[session_location.joinpath(session,'01 Processing_data','pupil_labs') for session in sessions ]
session_raw_list=[session_location.joinpath(session, '00 Raw_data','pupil_labs') for session in sessions ]

image_loc_list=[session_location.joinpath(session,'images_order') for session in sessions ]

for subject, recording_location in zip(sessions,session_loc_list):
    recording_folder=[record for record in os.listdir(recording_location)  if '00' in record]
    print(recording_folder)
    index_aux = list(map(lambda x: not('_' in x), recording_folder))
    recording_folder=list(compress(recording_folder,index_aux))
    #Load data in folders
    if len(recording_folder)>1:
        ValueError('Ambiguty in folder of experiment')
    recording_location=recording_location.joinpath(recording_folder[0],'exports')
    recording_location_raw=recording_location.joinpath(recording_folder[0],'exports')
    export_folder=[record for record in os.listdir(recording_location)  if '00' in record]
    if len(export_folder)>1:
        ValueError('Ambiguty in folder of exports')
    recording_location=recording_location.joinpath(export_folder[0])

    gaze_surf_csv=[record for record in os.listdir(recording_location.joinpath('surfaces'))  if 'gaze_positions' in record][0]

    print(f'Analixing session: {subject}')

    exported_pupil_csv = recording_location.joinpath('pupil_positions.csv')
    pupil_pd_frame = pd.read_csv(exported_pupil_csv,usecols=use_cols)
    exported_gaze_csv = os.path.join(recording_location,'gaze_positions.csv')
    gaze_pd_frame = pd.read_csv(exported_gaze_csv)
    annotations_csv = os.path.join(recording_location,'annotations.csv')
    annotations_pd = pd.read_csv(annotations_csv)
    surface_gaze_csv = os.path.join(recording_location, 'surfaces',gaze_surf_csv)
    surfaces_pd_frame = pd.read_csv(surface_gaze_csv)


['ES0001_S020_Script.txt', '001']
Analixing session: ES0001_S020_PC_EM
['ES0001_S022_Script.txt', '003']
Analixing session: ES0001_S022_PC_EM


### Filter data that is inside the surface
#### Due to the fact that te gaze timesatmp of the surface is obtained by combining data from both pupils we cant just merge by the timestamp. We have to read and duplicate values



### Cut all data by annotations of interest

In [30]:
# pupil_pd_frame=final_df

from IPython.display import display

event_initial=annotations_pd['label'].values[0]
event_final=annotations_pd['label'].values[-1]

initial_anotation,_,_=cm.extract_annotations_timestamps(event_initial,'label',annotations_pd)
end_anotation=annotations_pd.iloc[-1]
display(initial_anotation)
display(end_anotation)

pupil_pd_frame=cm.filter_rows_by_temporal_values(
         dataframe=pupil_pd_frame,
         time_column='pupil_timestamp',
         ini_value=initial_anotation['timestamp'].values[0],
         end_value=end_anotation['timestamp']
         )

,index,timestamp,label,duration
0,6231,21605.402144,blank_0,NaN


index                  38467
timestamp       22145.537089
label        EndOfExperiment
duration                 NaN
Name: 162, dtype: object

## Data cleaning
Extracting pupil 3d data for analysis:
1) Select eye that has a better signal
2) Clean blinks from data
3) Asses that quality

Now select eye with better data quality 

In [58]:
def extract_eye_data(pupil_df:pd.DataFrame,eye_id: str = 'best',
               method: str = 'pye3d 0.3.0 real-time',confidence_thr: float=0.95)-> pd.DataFrame:
    # TODO: Refactor function in smaller chunks-- clean code
    """Function to extract pupil data for eye of interest. 
    Left eye, right eye or best eye can be extracted for further analysis

    Args:
        pupil_df (pd.DataFrame): pupil pandas dataframe
        eye_id (str, optional): _description_. Defaults to 'best'.
        method (str, optional): _description_. Defaults to 'pye3d 0.3.0 real-time'.
        confidence_thr: Confidence thershold for data qualiltu filtering. Deafults to 0.95

    Raises:
        ValueError: a string must be put to indicate if we are using left, right eye or best

    Returns:
        pd.DataFrame: pandas dataframe with pupil data of interest
    """
    pupil_df = pupil_df.loc[pupil_df.method.str.contains(method)]

    pupil_left_eye=pupil_df.loc[(pupil_pd_frame['eye_id']==1) & (pupil_pd_frame['confidence']>=confidence_thr)]
    pupil_right_eye=pupil_df.loc[(pupil_pd_frame['eye_id']==0) & (pupil_pd_frame['confidence']>=confidence_thr)]

    blinks_left_eye=pupil_df.loc[(pupil_pd_frame['eye_id']==1) & (pupil_pd_frame['confidence']< confidence_thr)]
    blinks_right_eye=pupil_df.loc[(pupil_pd_frame['eye_id']==0) & (pupil_pd_frame['confidence']< confidence_thr)]


    ##  Explore blinks and Asses data quality, are there many regions with data gaps? 
    parts = list(recording_location.parts)
    parts=parts[0:-2]
    parts=Path(*parts)
    f = open(parts.joinpath('info.player.json'))
    data = json.load(f)
    time_whole_recording=data['duration_s']
    time_roi_left=abs(pupil_left_eye['pupil_timestamp'].values[-1]-pupil_left_eye['pupil_timestamp'].values[0])
    time_roi_right=abs(pupil_right_eye['pupil_timestamp'].values[-1]-pupil_right_eye['pupil_timestamp'].values[0])

    pupil_sampling_freq_left=(pupil_left_eye.shape[0]+blinks_left_eye.shape[0])/time_roi_left
    pupil_sampling_freq_right=(pupil_right_eye.shape[0]+blinks_right_eye.shape[0])/time_roi_right 
    print(f'time of the whole recording is {time_whole_recording}')
    print(f'time after anottaion cutting {time_roi_left:.2f}s for left eye and {time_roi_right:.2f}s for right eye')
    print('sampling frequency for right eye is {}'.format(pupil_sampling_freq_right)) 
    print('sampling frequency for left eye is {}'.format(pupil_sampling_freq_left)) 
    # Closing file
    f.close()

    ### Asses data quality
    total_blinks_left=blinks_left_eye.shape[0]/pupil_sampling_freq_left
    total_blinks_right=blinks_right_eye.shape[0]/pupil_sampling_freq_right
    print(f'Total blinks time left {total_blinks_left:.2f}s and right {total_blinks_right:.2f}s of a total of {time_roi_left:.2f}s left and {time_roi_right:.2f}s right')
    print(f'Total blinks time left {total_blinks_left/time_roi_left:.2%} ')
    print(f'Total blinks time right {total_blinks_right/time_roi_right:.2%} ')

    if eye_id == 'left':
        pupil_df = pupil_df[pupil_df.eye_id == 1]
    elif eye_id == 'right':
        pupil_df = pupil_df[pupil_df.eye_id == 0]
    elif eye_id == 'best':
        best = pupil_df.groupby(['eye_id'])['confidence'].mean()
        display(best)
        best=best.idxmax()
        pupil_df = pupil_df[pupil_df.eye_id == best]
        eye='left'if best==1 else 'right'
        print(f'data from {eye} eye was selected')
    else:
        raise ValueError('Eye must be "left", "right" or "best".')
    print('Loaded {} samples'.format(len(pupil_df)))
    return pupil_df


In [59]:
pupil_df=extract_eye_data(pupil_df=pupil_pd_frame,
                          eye_id='best',
                          method='pye3d 0.3.0 real-time')

time of the whole recording is 668.9416070000007
time after anottaion cutting 540.13s for left eye and 540.13s for right eye
sampling frequency for right eye is 123.92823997814095
sampling frequency for left eye is 123.93008313418737
Total blinks time left 73.78s and right 62.46s of a total of 540.13s left and 540.13s right
Total blinks time left 13.66% 
Total blinks time right 11.56% 


eye_id
0    0.931343
1    0.923288
Name: confidence, dtype: float64

data from right eye was selected
Loaded 66937 samples
